In [4]:
import pandas as pd

data = pd.read_excel("Datahub_Agri_Latest.xlsx", sheet_name="Database")

# Unpivot Agri Dataset

In [7]:

import pandas as pd
import sqlite3

# Step 1: Read the data
data = pd.read_excel("Datahub_Agri_Latest.xlsx", sheet_name="Database")

# Step 2: Check for a typo and fix the column name if necessary
if 'Latiude' in data.columns:
    data = data.rename(columns={'Latiude': 'Latitude'})  # Fix the typo for Latitude

# Step 3: Define the unit for each indicator
indicator_units = {
    'Area Planted': 'Ha',
    'Area Harvested': 'Ha',
    'Quantity Harvested': 'T',
    'Yield': 'T/Ha',
    'Smallholder Livestock': 'Number',
    'Livestock Farming Industry': 'Number',
    'Total Number of Livestock': 'Number',
    'Quantity': 'Kg',
    'Value': 'USD',
    'Share(%)': '%',
    'Growth(%)': '%',
    'Products': 'N/A',
    'Famers': 'Number',
    'Associates': 'Number',
    'SMEs': 'Number',
    'Company': 'Number'
}

# Step 4: Unpivot the data using pd.melt()
unpivoted_data = data.melt(
    id_vars=[
        'Series Code', 'Series Name', 'Sector', 'Sub-Sector (1)', 'Sub-Sector (2)', 
        'Province', 'Year', 'Source', 'Latitude', 'Longitude', 'Pro code', 
        'Markets'
    ], 
    value_vars=[
        'Area Planted', 'Area Harvested', 'Quantity Harvested', 'Yield', 
        'Smallholder Livestock', 'Livestock Farming Industry', 'Total Number of Livestock', 
        'Livestock Production', 'Famers', 'Associates', 'SMEs', 'Company', 
        'Quantity', 'Value', 'Share(%)', 'Growth(%)'
    ], 
    var_name='Indicator', 
    value_name='Indicator Value'
)

# Step 5: Add the 'Indicator Unit' column using the indicator_units dictionary
unpivoted_data['Indicator Unit'] = unpivoted_data['Indicator'].map(indicator_units)
unpivoted_data['Indicator Value'] = unpivoted_data['Indicator Value'].replace(['', ' ', 'N/A'], pd.NA)
unpivoted_data = unpivoted_data.dropna(subset=['Indicator Value'])
unpivoted_data['Year'] = unpivoted_data['Year'].astype(int)
unpivoted_data = unpivoted_data.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# Optionally, you can save the unpivoted data back to a new Excel file
unpivoted_data.to_excel("Unpivoted_Datahub_Agri_Latest.xlsx", index=False)


# Step 6: Connect to SQLite database
conn = sqlite3.connect("data.db")
table_name = "agriculture_data"
unpivoted_data.to_sql(table_name, conn, if_exists='replace', index=False)
conn.close()

C:\Users\Mr.Whatanak\AppData\Local\Temp\ipykernel_14832\991375240.py:54: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  unpivoted_data = unpivoted_data.applymap(lambda x: x.strip() if isinstance(x, str) else x)


# Unpivot Economic Dataset

In [12]:

import pandas as pd

# Step 1: Read the data
data = pd.read_excel("Datahub_Agri_Latest.xlsx", sheet_name="Sheet1")

# Step 2: Check for a typo and fix the column name if necessary
if 'Latiude' in data.columns:
    data = data.rename(columns={'Latiude': 'Latitude'})  # Fix the typo for Latitude

# Step 3: Define the unit for each indicator
indicator_units = {
    'Quantity': 'Kg',
    'Value': 'USD',
    'Share(%)': '%',
    'Growth(%)': '%'
}

# Step 4: Unpivot the data using pd.melt()
unpivoted_data = data.melt(
    id_vars=[
        'Series Code', 'Series Name', 'Sector', 'Sub-Sector (1)', 'Sub-Sector (2)', 
        'Province', 'Year', 'Source', 'Latitude', 'Longitude',
        'Products', 'Markets'
    ], 
    value_vars=[
        'Quantity', 'Value', 'Share(%)', 'Growth(%)'
    ], 
    var_name='Indicator', 
    value_name='Indicator Value'
)

# Step 5: Add the 'Indicator Unit' column using the indicator_units dictionary
unpivoted_data['Indicator Unit'] = unpivoted_data['Indicator'].map(indicator_units)

unpivoted_data['Indicator Value'] = unpivoted_data['Indicator Value'].replace(['', ' ', 'N/A'], pd.NA)
unpivoted_data = unpivoted_data.dropna(subset=['Indicator Value'])
unpivoted_data['Year'] = pd.to_numeric(unpivoted_data['Year'], errors='coerce')
unpivoted_data['Year'] = unpivoted_data['Year'].apply(lambda x: int(x) if not pd.isna(x) else x)
unpivoted_data = unpivoted_data.applymap(lambda x: x.strip() if isinstance(x, str) else x)


# Step 6: Strip leading and trailing spaces from text columns
text_columns = [
    'Series Name', 'Sector', 'Sub-Sector (1)',
    'Source', 'Products', 'Markets', 'Indicator', 'Indicator Unit'
]

for col in text_columns:
    unpivoted_data[col] = unpivoted_data[col].str.strip()
    
# Optionally, you can save the unpivoted data back to a new Excel file
unpivoted_data.to_excel("Unpivoted_Datahub_Economic.xlsx", index=False)

# Step 6: Connect to SQLite database
conn = sqlite3.connect("data.db")
table_name = "economic_data"
unpivoted_data.to_sql(table_name, conn, if_exists='replace', index=False)
conn.close()


C:\Users\Mr.Whatanak\AppData\Local\Temp\ipykernel_14832\4267086297.py:39: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  unpivoted_data = unpivoted_data.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [14]:
import sqlite3
import pandas as pd

# Step 1: Connect to the SQLite database
conn = sqlite3.connect("data.db")

# Step 2: Fetch the list of tables in the database (to confirm the table exists)
print("List of tables in the database:")
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
print(tables)

# Step 3: Read data from the table
table_name = "economic_data"

try:
    # Check if the table exists and load its content
    print(f"\nPreview of data from the '{table_name}' table:")
    data = pd.read_sql_query(f"SELECT * FROM {table_name} LIMIT 1000;", conn)
    print(data)
except Exception as e:
    print(f"Error: {e}. The table '{table_name}' might not exist.")

# Step 4: Close the connection
conn.close()


List of tables in the database:
               name
0  agriculture_data
1     economic_data

Preview of data from the 'economic_data' table:
    Series Code                   Series Name    Sector Sub-Sector (1)  \
0          None  Export, by exported products  Economic          Trade   
1          None  Export, by exported products  Economic          Trade   
2          None  Export, by exported products  Economic          Trade   
3          None  Export, by exported products  Economic          Trade   
4          None  Export, by exported products  Economic          Trade   
..          ...                           ...       ...            ...   
995        None  Total trade volume by market  Economic          Trade   
996        None  Total trade volume by market  Economic          Trade   
997        None  Total trade volume by market  Economic          Trade   
998        None  Total trade volume by market  Economic          Trade   
999        None  Total trade volume by market